# Explicação sobre o script de Experimento do LDA com Regressão Logística
Este notebook contém explicações sobre as etapas utilizadas para se treinar e calcular as métricas de qualidade dos modelos LDA e Regressão Logística, sendo utilizados com o objetivo de encontrar similaridade entre dois textos. Este notebook foi feito considerando apenas uma combinação de parâmetros, para executar todo o experimento seria necessário ter um loop executando estas etapas para todas as combinações de parâmetros

## Objetivo do Experimento

A ideia do experimento é calcular a similaridade entre dois documentos e classificá-las em classes de similaridade.
Para calcular a similaridade, precisamos transformar os documentos para um formato numérico. O vetorizador e o modelo LDA são utilizados para isso.

O vetorizador transforma o texto em um vetor de números, com os quais são possíveis fazer cálculos. Cada documento vira um vetor de números, em que cada posição representa uma palavra dentro do conjunto de todas as palavras contidas em todos os documentos. O LDA reduz o tamanho desse vetor para um tamanho bem menor, agrupando as palavras em grupos. Neste novo vetor de números, ao invés de cada posição representar uma palavra, cada posição representa um grupo de palavras.

Depois que os documentos são vetorizados, é possível fazer o cálculo de similaridade entre documentos.
Feito o cálculo da similaridade, passamos para a regressão logistica aprender quais distâncias pertencem a cada nível de similaridade

## Carregar dados de treino e de teste
Os dados de treino e de teste são um csv que contém, em cada linha, os IDs de um par de casos e o nível de similaridade entre eles. Queremos utilizar as distâncias entre os pares e as classes de similaridade para que a regressão logística aprenda quais similaridades pertencem a cada classe.
As etapas de vetorização, executar LDA são necessárias para se calcular as distâncias.
A leitura dos dados de treino e teste é realizada antes porque os dados de treino e teste são os mesmos, independente dos parâmetros. E, por causa disso, essa etapa só é executada uma vez

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
dados = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/data/bg_report_pares.csv')
dados_text = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/data/bg_report.csv')

dados = dados.sample(frac=1).reset_index(drop=True)

dados_treino = dados.sample(frac=0.8).reset_index(drop=True)#
dados_teste = dados.drop(dados_treino.index)
print(len(dados_treino))
print(len(dados_teste))
print(len(dados))

75759
18940
94699


In [ ]:
#dados_treino = pd.read_csv('./dados/dados_treino_teste/dados_treino.csv')
#dados_teste = pd.read_csv('./dados/dados_treino_teste/dados_teste.csv')
#print(len(dados_treino))
#print(len(dados_teste))

In [ ]:
dados_treino.dtypes

Issue_id            int64
Duplicated_issue    int64
label               int64
dtype: object

In [ ]:
dados_teste.dtypes

Issue_id            int64
Duplicated_issue    int64
label               int64
dtype: object

In [ ]:
dados_treino.head()

,Issue_id,Duplicated_issue,label
0,528102,266593,0
1,797731,511601,0
2,414287,615052,0
3,633680,214836,0
4,241116,355772,0


In [ ]:
dados_teste.head()
dados_treino.shape

(75759, 3)

In [ ]:
dados_teste.shape

(18940, 3)

## Parâmetros
Esta é a combinação de parâmetros a ser utilizada neste exemplo:

In [ ]:
parametros_vetorizador = {
        "cenario": "pp1",
        "tipo_vetorizador": "tf",
        "min_df": 1,
        "max_df": 1.0,
        "ngram_range": (1,1)
}   
parametros_modelo = {
        "n_components": 10,
        "alfa": 0.05,
        "beta": 0.01
}
parametros_regr_log = {
    "tipo_distancia": "cosseno",
    "metodo_balanceamento": "smote"
}

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, SnowballStemmer, WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

Preprocessando os dados

In [ ]:
#Lower case transformation
dados_text.loc[:,"Description"] = dados_text.Description.apply(lambda x : str.lower(x))

In [ ]:
#Tokenizer
dados_text.loc[:,"Description"] = dados_text.Description.apply(lambda x : word_tokenize(x))

In [ ]:
#Removing stopwords
stop_words = set(stopwords.words("english"))
dados_text.loc[:,"Description"] = dados_text.Description.apply(lambda x : [word for word in x if word not in stop_words])

In [ ]:
#Lemmatizing 
lemmatizer = WordNetLemmatizer()
dados_text.loc[:,"Description"] =  dados_text.Description.apply(lambda x : [lemmatizer.lemmatize(word) for word in x ])

## Lendo os textos pré-processados dos RDFs dos casos
O arquivo a ser lido varia de acordo com o cenário contido nos parâmetros

In [ ]:
cenario = parametros_vetorizador.get('cenario')
textos_preprocessados = dados_text
textos_preprocessados.dtypes

Issue_id        int64
Description    object
dtype: object

In [ ]:
textos_preprocessados.head()

,Issue_id,Description
0,10954,"[dialup, property, profile, exposed, prefs, pa..."
1,14871,"[please, add, match, whole, word, option, brow..."
2,19118,"[would, really, like, plug-in, manager, browse..."
3,54746,"[language, encoding, listed, seemingly, random..."
4,56892,"[bugzilla, helper, :, ;, user-agent, :, mozill..."


In [ ]:
dados_text.loc[:,"Description"] = dados_text.Description.apply(lambda x : str(x))

In [ ]:
textos_preprocessados.head()

,Issue_id,Description
0,10954,"['dialup', 'property', 'profile', 'exposed', '..."
1,14871,"['please', 'add', 'match', 'whole', 'word', 'o..."
2,19118,"['would', 'really', 'like', 'plug-in', 'manage..."
3,54746,"['language', 'encoding', 'listed', 'seemingly'..."
4,56892,"['bugzilla', 'helper', ':', ';', 'user-agent',..."


## Vetorizar Documentos

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.tokenize import RegexpTokenizer

In [ ]:
class VetorizadorTF:
    def __init__(self, frequencia_minima=1, frequencia_maxima=1.0, escala_ngramas=(1, 1)):
        self.regex = (r'\(?\d+\)?\s\d{4,5}\-\d{4}|\d{4,5}\-\d{4}|\w\S+\w|\w+')

        self.tokenizador = RegexpTokenizer(self.regex)
        self.vetorizador = CountVectorizer(
            tokenizer=self.tokenizador.tokenize,
            min_df=frequencia_minima,
            max_df=frequencia_maxima,
            ngram_range=escala_ngramas
        )

    def vetoriza(self, documentos):
        return self.vetorizador.fit_transform(documentos)

class VetorizadorTFIDF:
    def __init__(self, frequencia_minima=1, frequencia_maxima=1.0, escala_ngramas=(1, 1)):
        self.regex = (r'\(?\d+\)?\s\d{4,5}\-\d{4}|\d{4,5}\-\d{4}|\w\S+\w|\w+')

        self.tokenizador = RegexpTokenizer(self.regex)
        self.vetorizador = TfidfVectorizer(
            tokenizer=self.tokenizador.tokenize,
            min_df=frequencia_minima,
            max_df=frequencia_maxima,
            ngram_range=escala_ngramas
        )

    def vetoriza(self, documentos):
        return self.vetorizador.fit_transform(documentos)

In [ ]:
vetorizador = VetorizadorTF
if parametros_vetorizador['tipo_vetorizador'] == 'tf':
    vetorizador = VetorizadorTF(
        frequencia_minima=parametros_vetorizador.get('min_df'),
        frequencia_maxima=parametros_vetorizador.get('max_df'),
        escala_ngramas=parametros_vetorizador.get('ngram_range')
    )
elif parametros_vetorizador['tipo_vetorizador'] == 'tfidf':
    vetorizador = VetorizadorTFIDF(
        frequencia_minima=parametros_vetorizador.get('min_df'),
        frequencia_maxima=parametros_vetorizador.get('max_df'),
        escala_ngramas=parametros_vetorizador.get('ngram_range')
    )

In [ ]:
docs_vetorizados = vetorizador.vetoriza(textos_preprocessados['Description'])

In [ ]:
print(docs_vetorizados[0])

  (0, 170247)	1
  (0, 308700)	1
  (0, 307928)	2
  (0, 186865)	1
  (0, 302758)	2
  (0, 293725)	2
  (0, 367621)	1
  (0, 178597)	1
  (0, 378357)	1
  (0, 265578)	1
  (0, 121085)	1
  (0, 384527)	1
  (0, 257446)	1
  (0, 296923)	1
  (0, 212177)	1
  (0, 257738)	1
  (0, 179730)	1
  (0, 175214)	1
  (0, 299106)	1
  (0, 180670)	1
  (0, 249632)	1
  (0, 360402)	1
  (0, 124189)	1
  (0, 196465)	1


In [ ]:
type(docs_vetorizados[0])

scipy.sparse.csr.csr_matrix

## Executar LDA

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
class LDA:
    def __init__(self, alfa, beta, numero_topicos=5):
        self.modelo = LatentDirichletAllocation(
            n_components=numero_topicos,
            doc_topic_prior=alfa,
            topic_word_prior=beta
        )

    def realizar_agrupamento(self, documentos_vetorizados):
        self.modelo.fit(documentos_vetorizados)
        return self.modelo.transform(documentos_vetorizados)

In [ ]:
lda = LDA(
    alfa=parametros_modelo.get('alfa'),
    beta=parametros_modelo.get('beta'),
    numero_topicos=parametros_modelo.get('n_components')
)

In [ ]:
matriz_doc_topico = lda.realizar_agrupamento(docs_vetorizados)

In [ ]:
matriz_doc_topico.shape

(114737, 10)

In [ ]:
matriz_doc_topico[0]

array([0.00181818, 0.00181818, 0.98363636, 0.00181818, 0.00181818,
       0.00181818, 0.00181818, 0.00181818, 0.00181818, 0.00181818])

## Calcular Distâncias

In [ ]:
from sklearn.metrics.pairwise import cosine_distances
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import manhattan_distances

def calcula_distancia_cossenos(vetor1, vetor2):
    return cosine_distances(vetor1, vetor2)


def calcula_distancia_euclidiana(vetor1, vetor2):
    return euclidean_distances(vetor1, vetor2)


def calcula_distancia_manhattan(vetor1, vetor2):
    return manhattan_distances(vetor1, vetor2)

### Calcular distância entre dois vetores (exemplo)
Os vetores foram escolhidos de forma arbitrária

In [ ]:
'''vetor_doc1 = matriz_doc_topico[0]
vetor_doc2 = matriz_doc_topico[1]
distancia_cosseno = calcula_distancia_cossenos(vetor_doc1, vetor_doc2)
distancia_manhattan = calcula_distancia_manhattan(vetor_doc1, vetor_doc2)
distancia_euclidiana = calcula_distancia_euclidiana(vetor_doc1, vetor_doc2)

print('Dist. via cossenos:', distancia_cosseno)
print('Dist. via euclidiana:', distancia_euclidiana)
print('Dist. via manhattan:', distancia_manhattan)'''

"vetor_doc1 = matriz_doc_topico[0]\nvetor_doc2 = matriz_doc_topico[1]\ndistancia_cosseno = calcula_distancia_cossenos(vetor_doc1, vetor_doc2)\ndistancia_manhattan = calcula_distancia_manhattan(vetor_doc1, vetor_doc2)\ndistancia_euclidiana = calcula_distancia_euclidiana(vetor_doc1, vetor_doc2)\n\nprint('Dist. via cossenos:', distancia_cosseno)\nprint('Dist. via euclidiana:', distancia_euclidiana)\nprint('Dist. via manhattan:', distancia_manhattan)"

> Lembrar de usar o reshape para formatar os dados antes de usar a função de cálculo de distâncias e o [0][0] para obter a distância em formato númerico

In [ ]:
vetor_doc1 = matriz_doc_topico[0].reshape(1, -1)
vetor_doc2 = matriz_doc_topico[1].reshape(1, -1)

distancia_cosseno = calcula_distancia_cossenos(vetor_doc1, vetor_doc2)
distancia_manhattan = calcula_distancia_manhattan(vetor_doc1, vetor_doc2)
distancia_euclidiana = calcula_distancia_euclidiana(vetor_doc1, vetor_doc2)

print('Dist. via cossenos:', distancia_cosseno[0][0])
print('Dist. via euclidiana:', distancia_euclidiana)
print('Dist. via manhattan:', distancia_manhattan)

Dist. via cossenos: 4.398283178219664e-05
Dist. via euclidiana: [[0.02792661]]
Dist. via manhattan: [[0.05298701]]


### Calcular distâncias para os pares no conjunto de treino e de testes

In [ ]:
print(textos_preprocessados['Issue_id'])

0          10954
1          14871
2          19118
3          54746
4          56892
           ...  
114732    955873
114733    955877
114734    955890
114735    955891
114736    955893
Name: Issue_id, Length: 114737, dtype: int64


In [ ]:
print(dados_treino.head())

   Issue_id  Duplicated_issue  label
0    528102            266593      0
1    797731            511601      0
2    414287            615052      0
3    633680            214836      0
4    241116            355772      0


In [ ]:
print(dados_treino['Issue_id'] == 852451)

0        False
1        False
2        False
3        False
4        False
         ...  
75754    False
75755    False
75756    False
75757    False
75758    False
Name: Issue_id, Length: 75759, dtype: bool


In [ ]:
index = textos_preprocessados.index
a = index[textos_preprocessados['Issue_id'] == 916768]
print(a)
print(len(a))

Int64Index([112671], dtype='int64')
1


In [ ]:
print(dados_treino[:1])

   Issue_id  Duplicated_issue  label
0    528102            266593      0


In [ ]:
 #print(textos_preprocessados.loc[textos_preprocessados['Issue_id'] == dados_treino['Issue_id']])


In [ ]:
def calcular_distancia_entre_par_de_casosX(row):
    # A partir do ID, descobrir o INDEX
    index_doc_query = textos_preprocessados.loc[textos_preprocessados['Issue_id'] == row['Issue_id']].index
    index_doc_resp = textos_preprocessados.loc[textos_preprocessados['Issue_id'] == row['Duplicated_issue']].index
    
    vetor_LDA_doc_query = matriz_doc_topico[index_doc_query].reshape(1, -1)
    vetor_LDA_doc_resp = matriz_doc_topico[index_doc_resp].reshape(1, -1)

    distancia = calcula_distancia_cossenos(
        vetor_LDA_doc_query,
        vetor_LDA_doc_resp
    )

    return 1 - distancia[0][0]

In [ ]:
def calcular_distancia_entre_par_de_casos(row):
  
    
    issue_id = row['Issue_id'] #Tenho que pegar o issue_id  do treino
    duplicated_issue = row['Duplicated_issue'] #Tenho que pegar o duplicated_issue do treino
  
    index = textos_preprocessados.index #Todos os índices
    
    index_id = index[textos_preprocessados['Issue_id'] == issue_id] #Tenho que achar o índice do issue_id no documento de texto
    index_duplicated = index[textos_preprocessados['Issue_id'] == duplicated_issue] #Tenho que achar o índice do duplicated no documento de texto

    #print("issue_id = {} \n duplicated_issue = {} \n index_id = {} \n index_duplicated = {} \n".format(issue_id, duplicated_issue, index_id, index_duplicated))
    if (len(index_id) == 0 or len(index_duplicated) == 0):
      return 0
    vetor_LDA_doc_query = matriz_doc_topico[index_id].reshape(1, -1)
    vetor_LDA_doc_resp = matriz_doc_topico[index_duplicated].reshape(1, -1)
    distancia = calcula_distancia_cossenos(
        vetor_LDA_doc_query,
        vetor_LDA_doc_resp
    )
    #print(distancia[0][0])
    return 1 - distancia[0][0]


#### Adicionar distâncias entre os pares de casos no conjunto de treino

In [ ]:
distancias_treino = dados_treino.apply(
    lambda linha: calcular_distancia_entre_par_de_casos(linha),
    axis=1
)
len(distancias_treino) == len(dados_treino)

True

In [ ]:
dados_treino['distancia'] = distancias_treino
dados_treino[:30]

,Issue_id,Duplicated_issue,label,distancia
0,528102,266593,0,0.933526
1,797731,511601,0,0.465730
2,414287,615052,0,0.051284
3,633680,214836,0,0.024764
4,241116,355772,0,0.019642
5,481545,395161,1,0.596610
6,513168,451687,0,0.482987
7,342458,354259,0,0.004090
8,705100,703514,0,0.295205
9,184653,653318,0,0.629902


In [ ]:
len(dados_treino)

75759

#### Adicionar distâncias entre os pares de casos no conjunto de teste

In [ ]:
distancias_teste = dados_teste.apply(
    lambda linha: calcular_distancia_entre_par_de_casos(linha),
    axis=1
)
len(distancias_teste) == len(dados_teste)

True

In [ ]:
dados_teste['distancia'] = distancias_teste
dados_teste[:20]

,Issue_id,Duplicated_issue,label,distancia
75759,949686,514377,0,0.357978
75760,440369,489570,0,0.869162
75761,303686,249150,1,0.885266
75762,945008,538704,0,0.004441
75763,597409,521939,0,0.392087
75764,324509,694000,0,0.014435
75765,727917,355538,0,0.483529
75766,440360,403699,0,0.774128
75767,477618,452270,0,0.396977
75768,861886,532765,0,0.009309


## Passar dados para a Regressão Logística
A regressão logística recebe o nível de semelhança e as distâncias calculadas. O objetivo deste modelo é encontrar um valor (ou um intervalo) de semelhança que separe uma classe de similaridade de outra

### Formatação dos dados
Tem que colocar `.values.ravel()` para formatar o `y_treino` e `y_teste`

In [ ]:
x_treino = dados_treino.loc[:, dados_treino.columns == 'distancia']
y_treino = dados_treino.loc[:, dados_treino.columns == 'label'].values.ravel()

In [ ]:
len(x_treino)

75759

In [ ]:
x_teste = dados_teste.loc[:, dados_teste.columns == 'distancia']
y_teste = dados_teste.loc[:, dados_teste.columns == 'label'].values.ravel()

### Balanceamento dos dados
Serve para gerar, de forma sintética, mais exemplos para que a regressão logística consiga aprender melhor os limites das classes de similaridade.

Pular essa etapa se o balanceamento for igual a `nenhum`

In [ ]:
from imblearn.over_sampling import ADASYN, RandomOverSampler, SMOTE

def balancear_dados(preditores_treino, predicao_treino, metodo, n_vizinhos=8):
    if metodo == 'adasyn':
        balanceador = ADASYN(n_neighbors=n_vizinhos)
    if metodo == 'smote':
        balanceador = SMOTE(k_neighbors=n_vizinhos)
    else:
        balanceador = RandomOverSampler()
      
    return balanceador.fit_resample(
        preditores_treino.values.reshape(-1, 1),
        predicao_treino
    )

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
metodo_balanceamento = parametros_regr_log.get('metodo_balanceamento')
if metodo_balanceamento != "nenhum":
    x_treino, y_treino = balancear_dados(
        x_treino,
        y_treino,
        metodo_balanceamento
    )

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
len(x_treino)

126366

### Executar Regressão Logística

In [ ]:
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import StratifiedKFold


from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import cross_val_score


def validacao_cruzada(nome_do_modelo, dic_parametros, preditores, predicoes):
    """Valida modelos usando conjuntos de parâmetros de teste, dados par
    a aprendizado e as respostas corretas.

    Utiliza combinações de valores passados como parâmetro no dicionário
    para criar modelos e aplica-los em uma função de validação cruzada,
     utilizando valores das distâncias de cada documento e as predicoes
    corretas.

    Parâmetros:
      nome_do_modelo (str): modelo de processamento
        "knn": Utiliza o módulo KNeighborsClassifier do SKLearn
        "regressao": Utiliza o módulo LogisticRegression do SKLearn
      dic_parametros (dict): dicionário de conjuntos de parametros espec
      ifico de cada modelo.
        exemplo:
          parametrosKnn = {
            'n_neighbors': [5, 6, 7, 8, 9, 10],
            'weights': ['uniform', 'distance'],
            'algorithm': ['brute', 'ball_tree', 'kd_tree']
          }
      preditores (list): lista de valores das distancias
      predicoes (list): lista de valores de resultados corretos
    """
    lista_combinacoes_parametros = combina_parametros(dic_parametros)

    kfold = StratifiedKFold(n_splits=10, shuffle=True)

    resultados = []
    for parametros in lista_combinacoes_parametros:

        instancia = instancia_modelo(nome_do_modelo, parametros)
        #aplica o metodo de validacao cruzada
        pontuacao = cross_val_score(instancia, preditores, predicoes,
                                    scoring='f1_macro', cv=kfold).mean()

        resultados.append(pontuacao)

    pos_melhor_resultado = resultados.index(max(resultados))

    resultado = resultados[pos_melhor_resultado]
    melhores_parametros = lista_combinacoes_parametros[pos_melhor_resultado]

    return resultado, melhores_parametros


def combina_parametros(dic_parametros):
    """Cria e retorna uma lista a partir de conjuntos de valores passado
    s em um dicionário.

    Parâmetros:
      dic_parametros (dict): dicionário com conjuntos de valores passado
    s como parametros.
    """
    combinacoes = list(ParameterGrid(dic_parametros))

    return combinacoes


def instancia_modelo(modelo, parametros):
    """Instancia e retorna um objeto a partir do nome do modelo a ser us
    ado e de um dicionário com seus parâmetros.

    Parâmetros:
      modelo (str): modelo de processamento
      parametros (dict): parâmetros que vão ser usados para instanciar o
    objeto a ser retornado
    """
    instancia = None
    if modelo == 'knn':
        instancia = instancia_classificador_knn(parametros['n_neighbors'],
                                                parametros['weights'],
                                                parametros['algorithm'])
    elif modelo == 'regressao':
        instancia = instancia_regressor_logistico(parametros['alfa'],
                                                  parametros['max_iter'])
    else:
        raise Exception('Modelo não existe para teste.')

    return instancia


def instancia_classificador_knn(k=5, pesos='uniform', algoritmo='brute',
                                metrica='minkowski'):
    """Instancia e retorna um objeto KNeighborsClassifier utilizando seu
    s parâmetros.
    """
    instancia = KNeighborsClassifier(
        n_neighbors=k,
        weights=pesos,
        algorithm=algoritmo,
        metric=metrica
    )

    return instancia


def instancia_regressor_logistico(alfa, iteracoes_maximas, resolvedor='lbfgs',
                                  multi_class='multinomial'):
    """Instancia e retorna um objeto LogisticRegression utilizando seu
    s parâmetros.
    """
    instancia = LogisticRegression(
        C=alfa,
        solver=resolvedor,
        multi_class=multi_class,
        max_iter=iteracoes_maximas
    )

    return instancia

#### Treinamento da Regressão Logística


Primeiro se executa a regressão com parâmetros abaixo e para encontrar a melhor combinação para o conjunto de treino utilizado. A própria regressão logística possui uma métrica interna que calcula a qualidade dos resultados e por isso ela consegue definir o melhor valor por si só

In [ ]:
parametros_fixos_regr_logis = {
    'alfa': [0.01, 0.5, 1.0],
    'max_iter': [100]
}

In [ ]:
melhor_pontuacao, melhores_parametros = validacao_cruzada(
    nome_do_modelo='regressao',
    dic_parametros=parametros_fixos_regr_logis,
    preditores=x_treino,
    predicoes=y_treino
)

`melhor_pontuacao` se refere à melhor combinação de parâmetros encontrada. Tendo os melhores valores para os parâmetros, treina-se a regressão logística

In [ ]:
modelo_regr_logis = instancia_regressor_logistico(
    alfa=melhores_parametros.get('alfa'),
    iteracoes_maximas=melhores_parametros.get('max_iter')
)
modelo_regr_logis.fit(x_treino, y_treino)
y_pred = modelo_regr_logis.predict(x_teste)

In [ ]:
y_pred[1]

O próximo passo agora é calcular as métricas de qualidade usando o `y_teste` e o `y_pred`.

O `y_teste` é a resposta correta, são as classificações que o modelo precisaria fazer para acertar todas.

O `y_pred` foram os resultados que o modelo previu, com base no treino realizado e no conjunto de testes passado.

## Cálculo das métricas de qualidade

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, silhouette_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix, multilabel_confusion_matrix

def calcula_silhueta(dados, predicoes, random_state=None, metrica='euclidean'):
    coeficiente = silhouette_score(
        dados,
        predicoes,
        random_state=random_state,
        metric=metrica
    )
    return coeficiente


def calcula_matriz_confusao(valor_verdadeiro, predicao, classes):
    return confusion_matrix(valor_verdadeiro, predicao,
                            labels=np.unique(classes))


def calcula_matriz_confusao_multiclasse(valor_verdadeiro, predicao, classes):
    return multilabel_confusion_matrix(valor_verdadeiro, predicao,
                                       labels=np.unique(classes))


def calcula_cobertura(valor_verdadeiro, predicao, classes, metodo):
    return recall_score(valor_verdadeiro, predicao, average=metodo,
                        labels=np.unique(classes), zero_division=0)


def calcula_acuracia(valor_verdadeiro, predicao):
    return accuracy_score(valor_verdadeiro, predicao, normalize=True)


def calcula_precisao(valor_verdadeiro, predicao, classes, metodo):
    return precision_score(valor_verdadeiro, predicao, average=metodo,
                           labels=np.unique(classes), zero_division=0)


In [ ]:
classes = [0, 1]
metodo = "weighted"
acuracia = calcula_acuracia(y_teste, y_pred)
precisao = calcula_precisao(y_teste, y_pred, classes, metodo)
precisao_por_classe = calcula_precisao(y_teste, y_pred, classes, metodo=None)
mat_confusao = calcula_matriz_confusao(y_teste, y_pred, classes)
mat_confusao_multiclasse = calcula_matriz_confusao_multiclasse(y_teste, y_pred, classes)
cobertura = calcula_cobertura(y_teste, y_pred, classes, metodo)
cobertura_por_classe = calcula_cobertura(y_teste, y_pred, classes, metodo=None)

In [ ]:
precisao_por_classe

In [ ]:
acuracia

In [ ]:
mat_confusao_multiclasse[0][0][1]

In [ ]:
print("true negatives is {} \n false negatives is  {} \n  true positives is  {} \n and false positives is  {} \n".format(mat_confusao_multiclasse[0][0][0], mat_confusao_multiclasse[0][1][0], mat_confusao_multiclasse[0][1][1], mat_confusao_multiclasse[0][0][1], ))

In [ ]:
print("true negatives is {} \n false negatives is  {} \n  true positives is  {} \n and false positives is  {} \n".format(mat_confusao[0][0], mat_confusao[1][0], mat_confusao[1][1], mat_confusao[0][1], ))

In [ ]:
print("true negatives is {} \n false negatives is  {} \n  true positives is  {} \n and false positives is  {} \n".format(mat_confusao_multiclasse[1][0][0], mat_confusao_multiclasse[1][1][0], mat_confusao_multiclasse[1][1][1], mat_confusao_multiclasse[1][0][1], ))

In [ ]:
precisao

## Salvar os Resultados Parciais

In [ ]:
resultados = {
    "cenario": [],            
    "tipo_vetorizador": [],
    "min_df": [],
    "max_df": [],
    "ngram_range": [],
    "n_components": [],
    "alfa": [],
    "beta": [],
    "tipo_distancia": [],
    "metodo_balanceamento": [],
    "melhor_pontuacao": [],
    "melhor_alfa": [],
    "acuracia": [],
    "precisao": [],
    "precisao_por_classe": [],
    "cobertura": [],
    "cobertura_por_classe": [],
    "mat_confusao": [],
    "mat_confusao_multiclasse": []
}

In [ ]:
for parametro in parametros_vetorizador.keys():
    resultados.get(parametro).append(
        parametros_vetorizador.get(parametro)
    )
for parametro in parametros_modelo.keys():
    resultados.get(parametro).append(
        parametros_modelo.get(parametro)
    )
for parametro in parametros_regr_log.keys():
    resultados.get(parametro).append(
        parametros_regr_log.get(parametro)
    )

In [ ]:
resultados.get('melhor_pontuacao').append(melhor_pontuacao)
resultados.get('melhor_alfa').append(melhores_parametros.get('alfa'))
resultados.get('acuracia').append(acuracia)
resultados.get('precisao').append(precisao)
resultados.get('precisao_por_classe').append(precisao_por_classe)
resultados.get('cobertura').append(cobertura)
resultados.get('cobertura_por_classe').append(cobertura_por_classe)
resultados.get('mat_confusao').append(mat_confusao)
resultados.get('mat_confusao_multiclasse').append(mat_confusao_multiclasse)

In [ ]:
resultados

Agora, é só criar um dataframe e salvar em um arquivo csv

In [ ]:
resultados_parciais = pd.DataFrame(resultados)
resultados_parciais.to_csv(
    './resultados/experimento_lda.csv',
    index=False
)

## Executar KNN
A etapa de separar os dados e fazer o balanceamento é igual, o que muda é o treinamento

In [ ]:
parametros_knn = {
    'n_neighbors': [5, 6, 7, 8, 9, 10],
    'weights': ['uniform', 'distance'],
    'algorithm': ['brute', 'ball_tree', 'kd_tree']
}

In [ ]:
def instancia_classificador_knn(k=5, pesos='uniform', algoritmo='brute',
                                metrica='minkowski'):
    """Instancia e retorna um objeto KNeighborsClassifier utilizando seu
    s parâmetros.
    """
    instancia = KNeighborsClassifier(
        n_neighbors=k,
        weights=pesos,
        algorithm=algoritmo,
        metric=metrica
    )

    return instancia

In [ ]:
melhor_pontuacao, melhores_parametros = validacao_cruzada(
    nome_do_modelo="knn", 
    dic_parametros=parametros_knn,
    preditores=x_treino,
    predicoes=y_treino
)

knn = instancia_classificador_knn(
    k=melhores_parametros['n_neighbors'],
    pesos=melhores_parametros['weights'],
    algoritmo=melhores_parametros['algorithm']    
)

knn.fit(x_treino, y_treino)
y_pred = knn.predict(x_teste)

A etapa de calcular as métricas de qualidade também é igual à da regressão logística

In [ ]:
classes = [0, 1]
metodo = "weighted"
acuracia = calcula_acuracia(y_teste, y_pred)
precisao = calcula_precisao(y_teste, y_pred, classes, metodo)
precisao_por_classe = calcula_precisao(y_teste, y_pred, classes, metodo=None)
mat_confusao = calcula_matriz_confusao(y_teste, y_pred, classes)
mat_confusao_multiclasse = calcula_matriz_confusao_multiclasse(y_teste, y_pred, classes)
cobertura = calcula_cobertura(y_teste, y_pred, classes, metodo)
cobertura_por_classe = calcula_cobertura(y_teste, y_pred, classes, metodo=None)

In [ ]:
precisao_por_classe

Se fosse pra salvar, ia precisar mudar a variável `resultados` de acordo com os parâmetros do knn.